In [ ]:
# Python packages

#!pip install import-ipynb
#!pip install git+https://github.com/patrick-kidger/torchcubicspline.git # Not strictly needed but kept just in case

import import_ipynb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torchvision import datasets
from torchvision import transforms
import Preprocess 
from torchcubicspline import(natural_cubic_spline_coeffs, 
                             NaturalCubicSpline)
from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
from scipy.optimize import newton,least_squares

In [ ]:
# Load of data, if the functionality in this module is wished be tested here.

# Swap files prefix and postfix names
prefix = ['ad','cd','nk','sw','dk','EU','US','uk','jy']
prefix = ['ad'] # kept here in case you only want to run the bootstrapping for currency.
postfix = ["1year.xlsx", "2year.xlsx", "3year.xlsx", "5year.xlsx", "10year.xlsx", "15year.xlsx", "20year.xlsx", "30year.xlsx"]

# Training data range
date_start = "2009-12-31"
date_end = "2023-01-01"

data_path = 'data/BloombergData/'

# Call to Preprocess module
clean_data_pre, _ = Preprocess.data_clean(Preprocess.init_cleaning(prefix,postfix,data_path,date_start,date_end))


#Dividing the input data to get it in decimal

clean_data = [x[1:]/100 for x in clean_data_pre]
clean_data = np.array(clean_data).astype(float)


# Subsection of data for efficency, remove if full range is desired.
clean_data = clean_data[75:95]

In [ ]:
# Coverage function for calculating time differences ie. Deltas (Not used)

def cvg(date, diff):
    date = datetime.strptime(date, "%Y-%m-%d")
    print(date)
    new_date = date + timedelta(days=diff)

    return new_date.strftime("%Y-%m-%d")


# Curve class, used to create continuous curves from x and y values, and doing a CubicSpline.
# example usages: new_curve = Curve(maturities, guess)
# both inputs are arrays of equal length

class Curve():
    def __init__(self, x_values, y_values):
        cs = CubicSpline(x_values, y_values)
        self.x_values = np.linspace(1,31,30* 365, endpoint=True)
    
        self.y_values = cs(self.x_values)


    def set_value(self, x, y):
        idx = np.abs(self.x_values - x).argmin()
        self.y_values[idx] = y

    def get_value(self, x):
        # Find the index of the nearest x value
        idx = np.abs(self.x_values - x).argmin()
        return self.y_values[idx]



#Discounting function takes a Curve and a maturity(time)
def discounting(curve, maturity): 
    return np.exp(-curve.get_value(maturity)*maturity)

def discounting_nocurve(curve, maturity):
    return np.exp(-curve[int(maturity)-1] * maturity)


# Forward Rate takes a curve, a time and a diff(delta) and uses the discounting function to get forward rate.
def forward_rate(curve,t, diff):
    return (1/diff) * ((discounting(curve,t)/discounting(curve,t+diff))  -1)


#swap rate paper function:

def swap_rate_torch_paper(curve, maturities):
    swap_rates = []
    for maturity in maturities:
        float_leg = 1 - discounting(curve, maturity)
        fixed_leg = np.sum([discounting(curve, t) for t in range(1, int(maturity) + 1)])
        swap_rate = float_leg / fixed_leg
        swap_rates.append(swap_rate)
    return swap_rates 


# Objective function used in optimizer, takes a init_guess, maturities array, observed data and diff(delta)
#creates a Curve of the guesses, and caculates the swap_rate from that curve, returns the difference of real vs calc.
 
def objective_fct(init_guess, maturities,data_swaps, diff):
    curve_init = Curve(maturities, init_guess)
    calculated_swaps = swap_rate_torch_paper(curve_init, maturities)
    return data_swaps- calculated_swaps 


# function for calling optimizer, saves result. takes an objective function, init_guess, maturities array, observed data and diff(delta)
def get_result(obj_func, init_guess, maturities, swaps, diff):

    result = least_squares(obj_func, init_guess, args=(maturities, swaps, diff))
    return result.x

In [ ]:
# Code block for calculating all swaps and yields for the full data set. 

# maturities of the input swaps
maturities = [1, 2, 3, 5,10, 15,20, 30]


# function for bootstrapping all curves in data, calss get_result(), objective_fct() and swap_rate_torch_paper()
def all_curves(data, maturities,diff):
    init_guess = np.array([0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01])
    yields = []
    all_swaps = []
    for i in range(len(data)):
        yield_i = get_result(objective_fct, init_guess, maturities, data[i],diff)
        yields.append(yield_i)

        all_swaps.append(swap_rate_torch_paper(Curve(maturities, yield_i), maturities))
    return yields, all_swaps



In [ ]:
# Test suite, not for real use.

# call to optimizer, returns all yield_curves in swaps
yield_curves,swaps = all_curves(clean_data, maturities, 1)

#the first yield curve, for plotting
test_curve = Curve(maturities,yield_curves[0]) 

plt.plot(test_curve.x_values, test_curve.y_values*100, c = "r") # zero coupon curve
plt.title("Zero Coupon Yield Curve")
plt.xlabel("Maturity")
plt.ylabel("Yield %")
plt.ylim(1,3.5)
plt.show()


# code for getting all forward_rate curves and zero coupon prices. Uses interpolation.
diff = 1
dis = []
x = np.linspace(0,30-diff,30* 365, endpoint=True)
forwards = []
for i in x:
    forwards.append(forward_rate(test_curve,i,diff))
    dis.append(discounting(test_curve, i))


# plot of original input swap rate and the one infered by the yield_curve constructed in the optimization.
plt.title("Swap Rate Curve")
plt.plot([1,2,3,5,10,15,20,30],np.array(swaps[0])*100, c="b", label = "Model Swap Quotes") #swap curve
plt.scatter([1,2,3,5,10,15,20,30],clean_data[0]*100, c="r", label="Market Swap Quotes")
plt.legend()
plt.ylim(1,3.5)
plt.xlabel("Maturity")
plt.ylabel("Swap Rate")
#plt.plot(x,forwards, c = "b")
#print(swaps[15])
#print(clean_data[15])


In [ ]:
# code for plotting centered softmax function

# Definition of centered softmax
def centered_softmax(x):
     return (1/(1+np.exp(-x)))- 0.5

# plot code
x = np.linspace(-5,5,100)
test = []
for i in x:
     test.append(centered_softmax(i))


fig, ax = plt.subplots()
ax.plot(x,test)
ax.spines['left'].set_position('zero')
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.set_ylim(-1,1)
ax.set_yticks([ -0.5, -1, 0.5, 1] )
ax.set_xticks([-2,-4,2,4])
